## 1.レース情報をスクレイピング

jupyter notebook で [ImportError: lxml not found, please install it]エラーが出た場合は [pip3 install lxml] でインストール後プロジェクトを再起動


In [ ]:
import pandas as pd
from tqdm import tqdm
import time
import csv
import numpy as np
from functions import Molding

url >> race_id=2022 09 01 11 01 : 年  競馬場(01:札幌,02:函館,03:福島,04:新潟,05:東京,06:中山,07:中京,08:京都,09:阪神,10:小倉) 何回(今年その競馬場でのレース回数) 何日目(何レース目)

In [ ]:
pd.set_option('display.max_rows',5)
pd.options.display.width = 200
pd.options.display.column_space = 100

スクレイピング

In [ ]:
# df = pd.read_pickle("1.7.df.pickle")
# already = df.index.unique()
# already = list(map(str, already))
# already

In [ ]:
nondata = []
def scrape_race_results(race_id_lists,par_race_results={}):
  race_result_dict = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in already:
    #   continue
    time.sleep(1)
    try:
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      race_result_dict[_race_id] = pd.read_html(url)[0]
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      nondata.append(_race_id)
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return race_result_dict

dictのkeyをrace_idに変換


In [ ]:
def get_house_data():
  race_id_lists = []

  for 年 in range(2021,2022):
    for 競馬場 in range(1,3): # 1,11
      for 回 in range(1,4): # 1,11
        for 回目 in range(1,4): # 1,11
          for レース回 in range(1,13): # 1,13
            race_id = f"{年}{str(競馬場).zfill(2)}{str(回).zfill(2)}{str(回目).zfill(2)}{str(レース回).zfill(2)}"
            race_id_lists.append(race_id)
            
  race_result = scrape_race_results(race_id_lists)
  for key in race_result.keys():
    race_result[key].index = [key] * len(race_result[key])
    
  results = pd.concat([race_result[key] for key in race_result.keys()],sort=False)
  return results

新規でdata取得するとき

In [ ]:
house_data = get_house_data()

取得できない数値

In [ ]:
nondata

追加でdata取得するとき

In [ ]:
house_data

csvで保存

In [ ]:
house_data.to_csv('1.2.test1.csv')
# house_data.to_csv('1.2.test2.csv')

## 2.データの自動修正

1.csvデータがうまく取得できていない箇所を[サイト](https://race.netkeiba.com/race/result.html?race_id=202102010203)からコピー

2.csvファイルにペースト

3.修正箇所の行番号を"arr"に入力

In [ ]:
# arr = [6,7,184,340,341,722,813,814]
arr = [6,7,238,262,319,320,323,322,73,319]
clean_index = list(map(lambda x: x-1,arr))
clean_index

In [ ]:
l,l_length,columns_len = Molding(clean_index,"1.2.test1.csv")

In [ ]:
add = np.array(l[1:]).reshape(l_length-1,columns_len)
df_add = pd.DataFrame(add, columns=l[0])
print(df_add)

df_add.to_csv('1.2.test1.csv',index=False)
# or
# df_add.to_csv('1.2.test2.csv',index=False)

In [ ]:
time.sleep(2)
df = pd.read_csv("1.2.test1.csv")
# or
# df = pd.read_csv("1.2.test2.csv")
# df

## 最終確認

In [ ]:
# pd.set_option('display.max_rows',None)

# df.columns
# print(df["着順"].value_counts())
# print(df["枠"].value_counts())
# print(df["馬番"].value_counts())
# print(df["馬名"].value_counts())
# print(df["性齢"].value_counts())
# print(df["斤量"].value_counts())
# print(df["騎手"].value_counts())
# print(df["タイム"].value_counts())
# print(df["着差"].value_counts())
# print(df["人気"].value_counts())
# print(df["単勝オッズ"].value_counts())
# print(df["後3F"].value_counts())
# print(df["コーナー通過順"].value_counts())
# print(df["厩舎"].value_counts())
# print(df["馬体重(増減)"].value_counts())

In [ ]:
# house_data[house_data["厩舎"] == "36.8"].index